### Run upon export from spreadsheet

In [22]:
import os

from astroquery.mast import Catalogs
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 50)


tces_file = '/mnt/tess/labels/tce_bls_instar+old.csv'
labels_file = '/mnt/tess/labels/labels_vetting_v1.csv'


tce_table = pd.read_csv(tces_file, header=0, low_memory=False).set_index('tic_id')
tce_table = tce_table.drop(columns=['Unnamed: 0'])
tce_table['Duration'] /= 24.0

joined_table = tce_table

joined_table = joined_table.reset_index()[[
    'tic_id', 'RA', 'Dec', 'Tmag', 'Epoc', 'Period', 'Duration',
    'Transit_Depth', 'Sectors', 'star_rad', 'star_mass', 'teff',
    'logg', 'SN', 'Qingress'
]]


labels_table = pd.read_csv(labels_file, header=0, low_memory=False)
labels_table = labels_table.drop(columns=['261262721'])
labels_table['tic_id'] = labels_table['TIC ID']

disps = ['e', 'p', 'n', 'b', 't', 'u']
users = ['ch', 'et', 'md', 'as', 'mk']

for d in disps:
    labels_table[f'disp_{d}'] = 0

def set_labels(row):
    a = ~row.isna()
    if a['Final']:
        row[f'disp_{row["Final"][0]}'] = 1
        row[f'disp_{row["Final"][1]}'] = 1
    else:
        for user in users:
            if a[user] and row[user]:
                row[f'disp_{row[user][0]}'] += 1
                row[f'disp_{row[user][1]}'] += 1
                        
    return row

labels_table = labels_table.apply(set_labels, axis=1)
labels_table = labels_table[['tic_id', 'Split'] + [f'disp_{d}' for d in disps]]

joined_table = joined_table.set_index('tic_id')
labels_table = labels_table.set_index('tic_id')
joined_table = joined_table.join(labels_table, on='tic_id', how='inner')


toi = pd.read_csv('/mnt/tess/labels/tce_toi_vetting_processed.csv', header=0, low_memory=False).set_index('tic_id')
toi['Duration'] /= 24.0

toi['Split'] = toi.apply(lambda r: 'train' if r['rand'] < 80 else 'test', axis=1)
toi = toi.drop(columns=['Unnamed: 0', 'rand'])

joined_table = joined_table.append(toi)


print(f'Total entries: {len(joined_table)}')
joined_table = joined_table[
    sum(joined_table[f'disp_{d}'] for d in disps) > 0
]
print(f'Total labeled entries: {len(joined_table)}')


all_table = joined_table

t_train = joined_table[joined_table['Split'] == 'train']
t_val = joined_table[joined_table['Split'] == 'val']
t_test = joined_table[joined_table['Split'] == 'test']
print(f'Split sizes. Train: {len(t_train)}; Valid: {len(t_val)}; Test: {len(t_test)}')

t_train = t_train.drop(columns=['Split'])
t_val = t_val.drop(columns=['Split'])
t_test = t_test.drop(columns=['Split'])
all_table = all_table.drop(columns=['Split'])

t_train.to_csv('/mnt/tess/astronet/tces-vetting-v2-train.csv')
t_val.to_csv('/mnt/tess/astronet/tces-vetting-v2-val.csv')
t_test.to_csv('/mnt/tess/astronet/tces-vetting-v2-test.csv')
all_table.to_csv('/mnt/tess/astronet/tces-vetting-all.csv')

,RA,Dec,Tmag,Epoc,Period,Duration,Transit_Depth,Sectors,star_rad,star_mass,teff,logg,SN,Qingress,disp_e,disp_p,disp_n,disp_b,disp_t,disp_u
count,674.000000,674.000000,674.000000,6.740000e+02,674.000000,674.000000,674.000000,674.0,652.000000,637.000000,652.000000,637.000000,652.000000,652.000000,674.0,674.0,674.0,674.0,674.0,674.0
mean,164.311999,-14.929962,11.029822,2.458660e+06,13.614689,0.137279,9518.072700,-1.0,1.203495,1.006346,5628.015706,4.359673,1.203495,1.203495,0.0,1.0,0.0,0.0,0.0,0.0
std,106.338337,45.437605,1.700975,2.734027e+02,52.208575,0.115051,12919.508477,0.0,0.552141,0.361020,2092.176775,0.287457,0.552141,0.552141,0.0,0.0,0.0,0.0,0.0,0.0
min,0.187097,-88.568464,4.627800,2.457050e+06,0.257920,0.019667,90.000000,-1.0,0.147000,0.141690,2949.000000,3.449670,0.147000,0.147000,0.0,1.0,0.0,0.0,0.0,0.0
25%,76.220306,-51.956200,10.011850,2.458440e+06,2.733207,0.083094,2579.000000,-1.0,0.824187,0.825000,5040.942500,4.172330,0.824187,0.824187,0.0,1.0,0.0,0.0,0.0,0.0
50%,148.883806,-27.588781,11.058600,2.458599e+06,4.027286,0.115854,7015.000000,-1.0,1.129070,1.014000,5705.900000,4.361640,1.129070,1.129070,0.0,1.0,0.0,0.0,0.0,0.0
75%,268.744106,29.827395,12.280875,2.458845e+06,7.838412,0.160760,12519.250000,-1.0,1.556358,1.170000,6175.000000,4.538480,1.556358,1.556358,0.0,1.0,0.0,0.0,0.0,0.0
max,359.900305,85.233216,15.194200,2.459381e+06,440.785000,0.904167,160000.000000,-1.0,3.399090,2.530000,50000.000000,5.802400,3.399090,3.399090,0.0,1.0,0.0,0.0,0.0,0.0


In [62]:
all_table.sample(3)

,RA,Dec,Tmag,Epoc,Period,Duration,Transit_Depth,Sectors,star_rad,star_mass,teff,logg,SN,Qingress,disp_e,disp_p,disp_n,disp_b,disp_t,disp_u
tic_id,,,,,,,,,,,,,,,,,,,,
301455720,118.323180,-69.338252,11.0774,1629.2,1.794918,0.093228,6070.0,20.0,1.14245,1.08,5923.00,4.35582,60.54950,0.42736,1,0,0,0,1,0
406236094,279.147029,-42.087705,11.4879,1655.17,4.140848,0.135778,4010.0,20.0,1.95551,NaN,6014.90,NaN,40.03969,0.21332,0,1,0,0,1,0
138294130,229.991336,36.229649,10.6506,18,0.000113,3.000000,8416.0,-1.0,1.59874,1.10,5981.01,4.07191,1.59874,1.59874,0,1,0,0,0,0


### Run once

In [23]:
import numpy as np
import pandas as pd

def clean_tois():
    toi = pd.read_csv('/mnt/tess/labels/toi_pc_instar.csv', header=0, low_memory=False)
    
    toi = toi[toi['instar'] == False]
    toi = toi[toi['Master Disposition'].isin(['P', 'VPC', 'KP'])]
    toi = toi[toi['planet_period'] < 99999]

    toi['tic_id'] = toi['star_tic']
    toi['RA'] = toi['star_ra']
    toi['Dec'] = toi['star_dec']
    toi['Tmag'] = toi['star_tmag']
    toi['Epoc'] = toi['planet_epoch']
    toi['Period'] = toi['planet_period']
    toi['Duration'] = toi['planet_tdur']
    toi['Transit_Depth'] = toi['planet_depth']
    toi['Sectors'] = -1.0
    toi['teff'] = toi['star_teff']
    toi['logg'] = toi['star_logg']
    toi['SN'] = toi['star_rad']
    toi['Qingress'] = toi['star_rad']
    toi['disp_p'] = 1
    toi['disp_e'] = 0
    toi['disp_n'] = 0
    toi['disp_b'] = 0
    toi['disp_t'] = 0
    toi['disp_u'] = 0
    
    toi = toi[[
        'tic_id',
        'RA',
         'Dec',
         'Tmag',
         'Epoc',
         'Period',
         'Duration',
         'Transit_Depth',
         'Sectors',
         'star_rad',
         'star_mass',
         'teff',
         'logg',
         'SN',
         'Qingress',
         'disp_e',
         'disp_p',
         'disp_n',
         'disp_b',
         'disp_t',
         'disp_u',
    ]]
    
    toi['rand'] = toi.apply(lambda r: np.random.randint(0, 100), axis=1)
    
    toi.to_csv('/mnt/tess/labels/tce_toi_vetting_processed.csv')